### 0.0 follow setup instructions

ℹ️ use [`pylcaio.yml`](https://github.com/michaelweinold/config_conda/blob/main/pylcaio.yml) to set up working conda environment.

⚠️ beware the estimated hardware requirements:

1. \>16GB RAM

⚠️ beware the following error messages:

1. [`AttributeError: 'IOSystem' object has no attribute 'Z' and then KeyError: 'PRO'`](https://github.com/michaelweinold/pylcaio_integration_with_brightway/issues/4) \
Caused by repeated execution of the `database_loader.combine_ecoinvent_exiobase()` and/or `lcaio_object.hybridize()` functions.

⚠️ `brightway` import [currently breaks `%autoreload` magic](https://github.com/brightway-lca/brightway2/issues/49)

### 0.1. imports
#### 0.1.1. regular imports

In [2]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

#### 0.1.2. local imports

In [3]:
from helpers import (
    read_ecoinvent_pickle,
    read_exiobase_pickle
)

#### 0.1.3. load configuration file

In [27]:
with open('config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

#### 0.1.4. load `pylcaio`

In [29]:
sys.path.append(os.path.join(Path.home(), config['pylcaio'])) # required for local import of pylcaio
import pylcaio

### 0.2. file paths
#### 0.2.1. directories

In [33]:
#%%capture
# home directory
print(path_dir_home := Path.home())
# input directory
print(path_dir_databases := os.path.join(path_dir_home, config['path_dir_databases']))
# output directories
print(path_dir_data := os.path.join(path_dir_home, config['path_dir_data']))
print(path_dir_pylcaio := os.path.join(path_dir_home, path_dir_data, config['path_dir_pylcaio']))
print(path_dir_pymrio := os.path.join(path_dir_home, path_dir_data, config['path_dir_pymrio']))
print(path_dir_e2m := os.path.join(path_dir_home, path_dir_data, config['path_dir_e2m']))

/home/weinold
/home/weinold/data_pylcaio_input
/home/weinold/data
/home/weinold/data/data_pylcaio
/home/weinold/data/data_pymrio
/home/weinold/data/data_e2m


#### 0.2.2. files

In [34]:
#%%capture
# databases
print(path_exiobase := os.path.join(path_dir_home, path_dir_databases, config['exiobase']))
print(path_dir_ecoinvent := os.path.join(path_dir_home, path_dir_databases, config['ecoinvent']))
# pylcaio output
print(pylcaio_database_loader_class_instance := os.path.join(path_dir_pylcaio, config['pylcaio_database_loader_class_instance']))
print(pylcaio_class_instance_before_hybrid := os.path.join(path_dir_pylcaio, config['pylcaio_class_instance_before_hybrid']))
print(pylcaio_class_instance_after_hybrid := os.path.join(path_dir_pylcaio, config['pylcaio_class_instance_after_hybrid']))
# pymrio output
print(pymrio_class_instance := os.path.join(path_dir_pymrio, config['pymrio_class_instance']))

/home/weinold/data_pylcaio_input/exiobase_monetary_pxp/IOT_2012_pxp.zip
/home/weinold/data_pylcaio_input/ecoinvent-3.5-cutoff
/home/weinold/data/data_pylcaio/pylcaio_database_loader_class_instance.pkl
/home/weinold/data/data_pylcaio/pylcaio_class_instance_before_hybrid.pkl
/home/weinold/data/data_pylcaio/pylcaio_class_instance_after_hybrid.pkl
/home/weinold/data/data_pymrio/pymrio_class_instance.pkl


In [35]:
!mkdir -p $path_dir_data
!mkdir -p $path_dir_e2m
!mkdir -p $path_dir_ecoinvent && cp -ru /srv/data/ecoinvent-3.5-cutoff/* $path_dir_ecoinvent
#!mkdir -p $path_exiobase && cp -ru /srv/data/exiobase_monetary_pxp/* $path_exiobase

mkdir: missing operand
Try 'mkdir --help' for more information.


### 1.1. read databases and create dataframes


In [10]:
load_from_disk: bool = False

#### 1.1.1 read Exiobase database and save pickle to disk

❔ creates pymrio.IOSystem class instance (collection of pd.DataFrames etc.) \
⏳ ~1min if `load_from_disk == False`

In [13]:
if load_from_disk == True:
    exiobase: pymrio.core.mriosystem.IOSystem = read_exiobase_pickle(pymrio_class_instance)
else:
    exiobase: pymrio.IOSystem = pymrio.parse_exiobase3(path_exiobase)
    with open(pymrio_class_instance, 'wb') as file_handle:    
        pickle.dump(obj = exiobase, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

#### 1.1.2 read ecoinvent

❔ creates e2m.Ecospold2Matrix class instance and writes dataframe to defined output directory in `pickle` format. \
⏳ ~12min if `load_from_disk == False`

In [1]:
#%%capture
print(name_project_e2m := 'ecoinvent_3_5_cutoff')
print(filename_project_e2m := os.path.join(name_project_e2m + 'Pandas_symmNorm.gz.pickle'))
print(path_dir_e2m_logs := os.path.join(path_dir_e2m, name_project_e2m, '_log'))
print(path_file_e2m_pickle := os.path.join(path_dir_e2m, filename_project_e2m))
print(pattern_e2m_characterization_db := '*.db')

ecoinvent_3_5_cutoff
ecoinvent_3_5_cutoffPandas_symmNorm.gz.pickle


NameError: name 'path_dir_e2m' is not defined

In [17]:
def delete_e2m_files(list_string: list) -> None:
    for i in list_string:
        !rm -rf $i
    pass


def copy_ecoinvent_from_shared_to_local(path: path_dir_ecoinvent) -> None:
    !mkdir -p $path_dir_ecoinvent && cp -ru /srv/data/ecoinvent-3.5-cutoff/* $path_dir_ecoinvent
    pass

In [18]:
if load_from_disk == True:
    ecoinvent: dict = read_ecoinvent_pickle(path_file_e2m_pickle)
else:
    delete_e2m_files(
        [
            path_dir_e2m,
            path_dir_e2m_logs,
            path_dir_ecoinvent,
            pattern_e2m_characterization_db
        ]
    )
    copy_ecoinvent_from_shared_to_local(path_dir_ecoinvent)
    parser = e2m.Ecospold2Matrix(
        sys_dir = path_dir_ecoinvent,
        project_name = name_project_e2m,
        out_dir = path_dir_e2m,
        #characterisation_file = path_e2m_char_file,
        positive_waste = False,
        nan2null = True)
    parser.ecospold_to_Leontief(
        fileformats = 'Pandas',
        with_absolute_flows=True)
    ecoinvent: dict = read_ecoinvent_pickle(path_file_e2m_pickle)

2022-09-10 12:24:43,782 - ecoinvent_3_5_cutoff - INFO - Ecospold2Matrix Processing
INFO:ecoinvent_3_5_cutoff:Ecospold2Matrix Processing
2022-09-10 12:24:43,784 - ecoinvent_3_5_cutoff - INFO - Current git commit: 2b78f39ac419e16a6721aa27a0bdb8204e1f0f3a
INFO:ecoinvent_3_5_cutoff:Current git commit: 2b78f39ac419e16a6721aa27a0bdb8204e1f0f3a
2022-09-10 12:24:43,785 - ecoinvent_3_5_cutoff - INFO - Project name: ecoinvent_3_5_cutoff
INFO:ecoinvent_3_5_cutoff:Project name: ecoinvent_3_5_cutoff
2022-09-10 12:24:43,785 - ecoinvent_3_5_cutoff - INFO - Unit process and Master data directory: /home/weinold/data_pylcaio_input/ecoinvent-3.5-cutoff
INFO:ecoinvent_3_5_cutoff:Unit process and Master data directory: /home/weinold/data_pylcaio_input/ecoinvent-3.5-cutoff
2022-09-10 12:24:43,786 - ecoinvent_3_5_cutoff - INFO - Data saved in: /home/weinold/data/data_e2m
INFO:ecoinvent_3_5_cutoff:Data saved in: /home/weinold/data/data_e2m
2022-09-10 12:24:43,787 - ecoinvent_3_5_cutoff - INFO - Replace Not-a-

### 2.1. main `pylcaio` functionality

In [19]:
database_loader: pylcaio.DatabaseLoader  = pylcaio.DatabaseLoader(
    lca_database_processed = ecoinvent,
    io_database_processed = exiobase,
    lca_database_name_and_version = 'ecoinvent3.5',
    io_database_name_and_version = 'exiobase3')

In [20]:
with open(pylcaio_database_loader_class_instance, 'wb') as file_handle:
    pickle.dump(obj = database_loader, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
lcaio_object: pylcaio.LCAIO = database_loader.combine_ecoinvent_exiobase(
    complete_extensions = False,
    impact_world = False,
    regionalized = False)

No path for the capital folder was provided. Capitals will not be endogenized


In [22]:
with open(pylcaio_class_instance_before_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
lcaio_object.hybridize(
    price_neutral_cut_off_matrix = 'STAM',
    capitals = False,
    priceless_scaling = True)

Indentifying Rest of World regions...
Updating electricity prices...
Calculating productions volumes...
Adjusting low production volume processes...
Extending inventory...
Building H matrix...


/home/weinold/github/pylcaio_integration_with_brightway/src/pylcaio.py:877: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.H = self.H.append([self.H] * (self.number_of_countries_IO + self.number_of_RoW_IO - 1))


Building geography concordance...
Filter H matrix...
Build Cut-off matrix...
Add processes with 'priceless scaling' to Cut-off matrix...


In [24]:
with open(pylcaio_class_instance_after_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

### 2.2. move files to `/srv/data` for use by everyone else

⚠️ this needs `sudo` rights, so best execute in the command line

In [29]:
sudo mkdir -p /srv/data/pylcaio_output/pickle/*
sudo cp -r /home/weinold/data_pylcaio_output/* /srv/data/pylcaio_output/pickle/*

pylcaio.LCAIO

### X.1 exiobase download

⚠️ only relevant for users with sudo rights

to download the monetary product-by-production versions of Exiobase:

1. download using pymrio (compare below)

In [ ]:
pymrio.download_exiobase3(
    storage_folder='/home/weinold',
    system="pxp",
    years=[2011]
)

2. move files to /srv/data using `sudo`

```
sudo mv ~/*pxp.zip /srv/data/exiobase_monetary_pxp
```